<a href="https://colab.research.google.com/github/fdamata/pucrj-machinelearning-mvp-ml/blob/main/mvp_sprint_02_fma_2025_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### MVP Machine Learning & Analytics

**Nome:** Fabiano da Mata Almeida<br>
**Matrícula:** 4052025000952<br>
**Dataset:** Pressão de Vapor da nafta.

**Nota sobre confidencialidade e descaracterização dos dados:**  
> Para garantir a confidencialidade e o respeito à privacidade, todos os dados utilizados neste estudo foram devidamente descaracterizados, não permitindo a identificação na sua unidade de medida original ou informações sensíveis.<br>
O uso desse dataset segue as boas práticas de ética em ciência de dados, assegurando que nenhuma informação pessoal ou confidencial seja exposta durante as análises.

# 1. Escopo, Objetivo e Definição do Problema

## 1.1. Contexto do problema e objetivo

O conjunto de dados **Pressão de Vapor da Nafta** contempla observações de uma corrente do processo de fracionamento do petróleo, contendo variáveis físico-químicas do processo e propriedades da corrente.

A variável alvo representa a pressão exercida pelo vapor em equilíbrio com sua fase líquida a uma temperatura específica. Essa propriedade caracteriza a volatilidade da corrente, impactando requisitos de armazenamento, segurança, transporte e conformidade regulatória.

**Objetivo:** Desenvolver um modelo preditivo para estimar a pressão de vapor da nafta com precisão, permitindo operação próxima ao limite superior da especificação sem comprometer a segurança.

*Nota: O conjunto de dados foi previamente analisado na SPRINT de Análise de Dados e Boas Práticas, permitindo EDA simplificada.*

## 1.2. Tipo de tarefa

Estudo de **regressão** com dados tabulares originados de sensores industriais e ensaios laboratoriais, aplicado à engenharia de processos na formulação de gasolina automotiva.

A tarefa envolve desenvolvimento de modelo supervisionado para predição contínua de propriedade físico-química crítica.

## 1.3. Valor para o negócio/usuário

**Ganho econômico:** A predição confiável permite incorporar frações mais pesadas do GLP à nafta, agregando valor significativo (GLP vale ~50% da gasolina por volume).

**Aplicações operacionais:**
- Otimização de parâmetros operacionais em tempo real
- Integração em sistemas de controle preditivo multivariável (MPC)
- Automação da gestão da propriedade

**Beneficiários:** Engenharia de Processos, Operações Industriais e Segurança Operacional, através de maior eficiência, previsibilidade e controle de qualidade.


# 2. Reprodutibilidade e ambiente

Esta seção consolida todas as importações de bibliotecas necessárias, definições das funções utilizadas e algumas configurações iniciais globais.

In [ ]:
# Bibliotecas básicas
import time, ast, sys, joblib, math, os, json, importlib
from copy import deepcopy
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import viridis
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import seaborn as sns

# Visualização
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate

# Análise estatística
from scipy.stats import norm, kstest
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Machine Learning - Model Selection and Evaluation
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler, RobustScaler, Normalizer
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.linear_model import LinearRegression,Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

!pip install -q catboost
!pip install -q scikit-optimize

from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import xgboost as xgb
from lightgbm import LGBMRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from skopt.callbacks import VerboseCallback, TimerCallback, DeltaYStopper
from skopt.plots import plot_objective, plot_convergence, plot_evaluations, plot_histogram, plot_objective_2D, plot_regret, plot_gaussian_process



# Configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Configurações de exibição
pd.options.display.float_format = '{:.4f}'.format  # Define o formato de exibição dos números float no pandas para quatro casas decimais
pd.set_option('display.expand_frame_repr', False)  # Não quebra a representação do dataframe
np.set_printoptions(precision=8, suppress=True, floatmode='maxprec') # Define o formato de exibição dos números float no numpy para oito casas decimais e sem notação científica


## 2.1 Definições prévias do problema

In [ ]:
# Definição do problema e inicialização de variáveis
PROBLEM_TYPE = "regressao"
SEED = 42
SPLIT = 0.3
CV_FOLDS = 10
METRIC_TO_OPT = 'r2'  # Pode ser 'r2', 'rmse', ou 'mae'

MINIMIZE_METRICS = {'rmse', 'mae', 'mse', 'msle', 'rmsle', 'mape', 'poisson', 'gamma', 'max_error', 'medae'}
MAXIMIZE_METRICS = {'r2', 'var', 'd2', 'explained_variance'}

## 2.2 Dependências
Eventual instalação de pacotes extras. <br>
*Manter o projeto enxuto*.

In [ ]:
# Exemplo: descomente o que precisar
# !pip install -q scikit-learn imbalanced-learn xgboost lightgbm catboost optuna
# !pip install -q pandas-profiling ydata-profiling
# !pip install -q matplotlib seaborn plotly
# !pip install -q statsmodels pmdarima

## 2.3. Funções Python
Definição de funções em Python. <br>
*Essa é uma boa prática de programação que facilita a leitura, manutenção e evolução do seu projeto.*

In [ ]:
# Declaração de funções

def teste_n(df, column_name, alpha=0.05):
    """
    Executa o teste de Kolmogorov-Smirnov para verificar se uma coluna do DataFrame segue uma distribuição normal.

    Parâmetros:
    -----------
    df : pandas.DataFrame
        DataFrame contendo os dados.
    column_name : str
        Nome da coluna a ser testada quanto à normalidade.
    alpha : float, opcional
        Nível de significância para o teste. O padrão é 0.05 (5%).

    Descrição:
    ----------
    - Normaliza os dados da coluna (subtrai a média e divide pelo desvio padrão).
    - Aplica o teste de Kolmogorov-Smirnov comparando com uma distribuição normal padrão.
    - Interpreta os resultados com base no p-valor e o nível de significância especificado.
    - Exibe uma mensagem informando se a distribuição pode ser considerada normal ou não.

    Retorno:
    --------
    tuple
        Uma tupla contendo (estatística do teste, p-valor).

    Exemplo de uso:
    --------------
    stat, p_valor = teste_n(df, 'pv_nafta')
    stat, p_valor = teste_n(df, 'pv_nafta', alpha=0.01)  # Usando significância de 1%
    """
    # Executar o teste de Kolmogorov-Smirnov - nesse caso em relação a uma distribuição normal
    stat, p_valor = kstest((df[column_name] - np.mean(df[column_name])) / np.std(df[column_name], ddof=1), 'norm')

    # Interpretar os resultados
    if p_valor > alpha:
        print("A amostra parece vir de uma distribuição normal (não podemos rejeitar a hipótese nula) p-valor:", f"{p_valor:.5f}")
    else:
        print("A amostra não parece vir de uma distribuição normal (rejeitamos a hipótese nula) p-valor:", f"{p_valor:.5f}")
    return float(stat), float(p_valor)

def calcula_corr(df):
    """
    Calcula e visualiza a matriz de correlação absoluta entre as variáveis de um DataFrame.

    Parâmetros:
    -----------
    df : pandas.DataFrame
        DataFrame contendo os dados para cálculo da correlação.

    Descrição:
    ----------
    - Remove valores NaN do DataFrame antes de calcular as correlações.
    - Calcula a matriz de correlação absoluta entre todas as variáveis.
    - Cria uma visualização interativa usando Plotly Express.
    - Aplica uma escala de cores Viridis para representar a intensidade das correlações.
    - Mostra os valores numéricos das correlações com duas casas decimais.

    Exemplo de uso:
    --------------
    calcula_corr(df)
    calcula_corr(df[selected_columns])  # Para um subconjunto de colunas
    """
    df_corr = df.dropna().corr().abs()

    fig = px.imshow(
        img    = df_corr,
        color_continuous_scale='Viridis',
        width  = 900, # caso não esteja visualizando todas as variáveis, altere esse valor
        height = 900, # caso não esteja visualizando todas as variáveis, altere esse valor
        text_auto = ".2f"  # Mostra os valores com 2 casas decimais
    )

    fig.update_traces(textfont_size=12)  # Altere o valor conforme desejado
    fig.show()

def pairplot_corr_hm(df, figsize=(12, 12), hist_bins=30, s=10, alpha=0.6):
    """
    Cria um pairplot onde a cor dos pontos é baseada na correlação absoluta entre variáveis
    usando uma paleta de cores Viridis.

    Parâmetros:
    -----------
    df : pandas DataFrame
        O DataFrame contendo os dados a serem plotados
    figsize : tuple, opcional
        Tamanho da figura (largura, altura) em polegadas
    hist_bins : int, opcional
        Número de bins para os histogramas na diagonal
    s : int, opcional
        Tamanho dos pontos nos gráficos de dispersão
    alpha : float, opcional
        Nível de transparência dos pontos (0-1)
    """

    # Obter a matriz de correlação absoluta
    corr_matrix = df.corr().abs()

    # Configurar a normalização de cores para a escala viridis
    norm = Normalize(vmin=0, vmax=1)

    # Obter as variáveis e o número de variáveis
    variables = df.columns
    n_vars = len(variables)

    # Criar a figura e os subplots
    fig, axes = plt.subplots(n_vars, n_vars, figsize=figsize)

    # Ajustar o espaçamento entre os subplots
    plt.subplots_adjust(wspace=0.2, hspace=0.2)

    # Criar os gráficos para cada par de variáveis
    for i, var1 in enumerate(variables):
        for j, var2 in enumerate(variables):
            ax = axes[i, j]

            # Remover os ticks dos eixos internos
            if i < n_vars - 1:
                ax.set_xticks([])
            if j > 0:
                ax.set_yticks([])

            # Se estamos na diagonal, plotar histograma
            if i == j:
                ax.hist(df[var1], bins=hist_bins, alpha=0.7, color='darkblue')
                ax.set_title(var1, fontsize=10)
            else:
                # Obter a correlação absoluta entre as variáveis
                corr_val = corr_matrix.loc[var1, var2]

                # Determinar a cor com base na correlação
                color = viridis(norm(corr_val))

                # Criar o gráfico de dispersão
                ax.scatter(df[var2], df[var1], s=s, alpha=alpha, color=color)

                # Adicionar a correlação como texto no gráfico
                ax.text(0.05, 0.95, f'|ρ|: {corr_val:.2f}',
                        transform=ax.transAxes, fontsize=8,
                        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

    # Adicionar os nomes das variáveis apenas nos eixos externos
    for i, var in enumerate(variables):
        axes[n_vars-1, i].set_xlabel(var, fontsize=10)
        axes[i, 0].set_ylabel(var, fontsize=10)

    # Adicionar uma barra de cores para referência
    cbar_ax = fig.add_axes([0.92, 0.3, 0.02, 0.4])  # [left, bottom, width, height]
    cb = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap="viridis"), cax=cbar_ax)
    cb.set_label('Correlação Absoluta |ρ|')

    plt.suptitle('Pairplot com Cores Baseadas na Correlação Absoluta', fontsize=16)
    plt.subplots_adjust(left=0.05, right=0.9, top=0.95, bottom=0.05, wspace=0.2, hspace=0.2)

def calcula_vif(df,target):
    """
    Calcula o Fator de Inflação da Variância (VIF) para identificar multicolinearidade em variáveis.

    Parâmetros:
    -----------
    df : pandas.DataFrame
        DataFrame contendo apenas as variáveis independentes para as quais se deseja calcular o VIF.

    Descrição:
    ----------
    - Adiciona uma constante ao DataFrame para o cálculo correto do VIF.
    - Calcula o VIF para cada variável usando a função variance_inflation_factor.
    - Ordena os resultados em ordem decrescente para identificar as variáveis mais problemáticas.
    - Exibe os resultados das 15 variáveis com maior VIF.

    # Retorno:
    # --------
    # pandas.DataFrame
    #     DataFrame contendo as variáveis e seus respectivos valores VIF.

    Interpretação:
    --------------
    - VIF = 1: Ausência de multicolinearidade
    - 1 < VIF < 5: Multicolinearidade moderada
    - 5 < VIF < 10: Multicolinearidade alta
    - VIF > 10: Multicolinearidade muito alta (problemática)

    Exemplo de uso:
    --------------
    vif_df = calcula_vif(df,target)
    """
    # Remove a coluna da variável target, se existir
    X = df.drop(columns=[target]) if target in df.columns else df.copy()
    X_with_const = sm.add_constant(X)  # Adicionando uma constante
    vif = pd.DataFrame()
    vif["Variable"] = X_with_const.columns
    vif["VIF"] = [variance_inflation_factor(X_with_const.values, i) for i in range(X_with_const.shape[1])]

    vif.set_index('Variable', inplace=True)
    # Imprimir VIF em ordem decrescente
    print("\nVIF das variáveis (ordem decrescente):\n")
    print(vif.query("Variable != 'const'").sort_values(by='VIF', ascending=False).head(15).T)

    # return vif

def plot_boxplot_pdf(df, lower_lim=None, upper_lim=None, n_cols=4):
    """
    Plota boxplot horizontal e PDF (histograma + curva normal) para todas as colunas numéricas do DataFrame.
    O layout é de múltiplas linhas e n_cols colunas de subplots: para cada coluna, boxplot em cima, PDF embaixo.

    Parâmetros:
    -----------
    df : pandas.DataFrame
        DataFrame contendo os dados.
    lower_lim : float, str ou None, opcional
        Limite inferior do eixo x. Se None ou 'auto', usa o mínimo dos dados.
    upper_lim : float, str ou None, opcional
        Limite superior do eixo x. Se None ou 'auto', usa o máximo dos dados.
    n_cols : int, opcional
        Número de colunas no layout dos subplots. O padrão é 4.

    Descrição:
    ----------
    - Para cada variável numérica, cria dois gráficos alinhados verticalmente:
      - Um boxplot horizontal no topo para visualizar a distribuição e outliers
      - Um histograma com curva normal teórica abaixo para visualizar a distribuição de frequência
    - Adiciona linhas de referência nos gráficos (média, mediana, ±3σ)
    - Permite ajustar os limites dos eixos manualmente ou automaticamente
    - Remove automaticamente valores NaN antes de plotar

    Exemplo de uso:
    --------------
    plot_boxplot_pdf(df)
    plot_boxplot_pdf(df, upper_lim=100, lower_lim=0)  # Define limites fixos para todas as variáveis
    plot_boxplot_pdf(df, n_cols=3)  # Altera o número de colunas no layout
    """

    numeric_cols = df.select_dtypes(include=[np.number]).columns
    n_vars = len(numeric_cols)
    n_rows = int(np.ceil(n_vars / n_cols))

    # Dobrar a altura dos plots da PDF (segunda linha de cada variável)
    height_ratios = []
    for _ in range(n_rows):
        height_ratios.extend([1, 4])  # boxplot:1, pdf:4

    fig, axes = plt.subplots(
        n_rows * 2,
        n_cols,
        figsize=(6 * n_cols, 5 * n_rows),
        gridspec_kw={'height_ratios': height_ratios}
    )

    axes = np.array(axes).reshape(n_rows * 2, n_cols)

    for idx, column in enumerate(numeric_cols):
        row = (idx // n_cols) * 2
        col = idx % n_cols
        data = df[column].dropna()

        # Use os valores reais dos dados para garantir que todos os outliers estejam visíveis
        data_min = data.min()
        data_max = data.max()

        # Se upper_lim/lower_lim forem fornecidos, use-os, senão use os valores reais dos dados
        if upper_lim is None or (isinstance(upper_lim, str) and upper_lim.lower() == 'auto'):
            x_upper = data_max
        else:
            try:
                x_upper = float(upper_lim)
            except (ValueError, TypeError):
                x_upper = data_max

        if lower_lim is None or (isinstance(lower_lim, str) and lower_lim.lower() == 'auto'):
            x_lower = data_min
        else:
            try:
                x_lower = float(lower_lim)
            except (ValueError, TypeError):
                x_lower = data_min

        # Para garantir que todos os pontos (inclusive outliers) sejam mostrados, defina os limites do eixo x
        # um pouco além dos valores mínimos e máximos reais dos dados
        margin = 0.02 * (data_max - data_min) if data_max > data_min else 1
        xlim_lower = data_min - margin
        xlim_upper = data_max + margin

        # Boxplot
        ax_box = axes[row, col]
        ax_box.boxplot(data, vert=False, patch_artist=True, widths=0.5, showfliers=True)
        ax_box.set_xlim(xlim_lower, xlim_upper)
        ax_box.set_yticks([])
        ax_box.set_xticklabels([])
        ax_box.set_title(f'Boxplot de {column}')

        # PDF (histograma + curva normal)
        ax_pdf = axes[row + 1, col]
        ax_pdf.hist(data, bins=30, color='lightblue', edgecolor='black', alpha=0.7, density=True, range=(xlim_lower, xlim_upper))

        if len(data) > 1:
            media = data.mean()
            std = data.std()
            x_grid = np.linspace(xlim_lower, xlim_upper, 200)
            y_norm = norm.pdf(x_grid, media, std)
            ax_pdf.plot(x_grid, y_norm, color='darkblue', lw=2, label='Normal')

            mediana = data.median()
            # Linhas estatísticas
            ax_box.axvline(media, color='blue', linestyle='-', label=f'Média: {media:.1f}')
            ax_box.axvline(mediana, color='green', linestyle='--', label=f'Mediana: {mediana:.1f}')
            ax_box.axvline(media + 3*std, color='orange', linestyle=':', label=f'+3σ: {(media + 3*std):.1f}')
            ax_box.axvline(media - 3*std, color='orange', linestyle=':', label=f'-3σ: {(media - 3*std):.1f}')

            ax_pdf.axvline(media, color='blue', linestyle='-', label=f'Média: {media:.1f}')
            ax_pdf.axvline(mediana, color='green', linestyle='--', label=f'Mediana: {mediana:.1f}')
            ax_pdf.axvline(media + 3*std, color='orange', linestyle=':', label=f'+3σ: {(media + 3*std):.1f}')
            ax_pdf.axvline(media - 3*std, color='orange', linestyle=':', label=f'-3σ: {(media - 3*std):.1f}')

        ax_pdf.set_xlim(xlim_lower, xlim_upper)
        ax_pdf.set_xlabel(column)
        ax_pdf.set_ylabel('Densidade')
        ax_pdf.set_title(f'PDF de {column}')
        ax_pdf.legend(fontsize=8, loc='upper left')

    # Remove subplots vazios
    total_plots = n_rows * n_cols
    for idx in range(n_vars, total_plots):
        for r in [0, 1]:
            fig.delaxes(axes[(idx // n_cols) * 2 + r, idx % n_cols])

    plt.tight_layout(h_pad=2.5)
    plt.show()

def plot_boxplot_pdf_indiv(df, column, lower_lim=None, upper_lim=None):
    """
    Plota boxplot horizontal e PDF (histograma + curva normal) para uma coluna numérica do DataFrame.

    Parâmetros:
    -----------
    df : pandas.DataFrame
        DataFrame contendo os dados.
    column : str
        Nome da coluna a ser plotada.
    lower_lim : float, str ou None, opcional
        Limite inferior do eixo x. Se None ou 'auto', usa o mínimo dos dados.
    upper_lim : float, str ou None, opcional
        Limite superior do eixo x. Se None ou 'auto', usa o máximo dos dados.
    """

    data = df[column].dropna()
    data_min = data.min()
    data_max = data.max()

    # Processamento dos limites
    if upper_lim is None or (isinstance(upper_lim, str) and upper_lim.lower() == 'auto'):
        x_upper = data_max
    else:
        try:
            x_upper = float(upper_lim)
        except (ValueError, TypeError):
            x_upper = data_max

    if lower_lim is None or (isinstance(lower_lim, str) and lower_lim.lower() == 'auto'):
        x_lower = data_min
    else:
        try:
            x_lower = float(lower_lim)
        except (ValueError, TypeError):
            x_lower = data_min

    # Margem para visualização
    margin = 0.02 * (data_max - data_min) if data_max > data_min else 1

    # Usar os limites definidos pelo usuário quando fornecidos
    xlim_lower = x_lower if lower_lim is not None and lower_lim != 'auto' else data_min - margin
    xlim_upper = x_upper if upper_lim is not None and upper_lim != 'auto' else data_max + margin

    fig, axes = plt.subplots(2, 1, figsize=(8, 6), gridspec_kw={'height_ratios': [1, 4]})

    # Boxplot
    ax_box = axes[0]
    ax_box.boxplot(data, vert=False, patch_artist=True, widths=0.5, showfliers=True)
    ax_box.set_xlim(xlim_lower, xlim_upper)
    ax_box.set_yticks([])
    ax_box.set_xticklabels([])
    ax_box.set_title(f'Boxplot de {column}')

    # PDF (histograma + curva normal)
    ax_pdf = axes[1]

    # Ajustar o range do histograma para os limites definidos
    hist_range = (xlim_lower, xlim_upper)
    ax_pdf.hist(data, bins=30, color='lightblue', edgecolor='black', alpha=0.7, density=True, range=hist_range)

    if len(data) > 1:
        media = data.mean()
        std = data.std()
        mediana = data.median()

        # Usar os limites definidos para o grid da curva normal
        x_grid = np.linspace(xlim_lower, xlim_upper, 200)
        y_norm = norm.pdf(x_grid, media, std)
        ax_pdf.plot(x_grid, y_norm, color='darkblue', lw=2, label='Normal')

        # Linhas estatísticas
        ax_box.axvline(media, color='blue', linestyle='-', label=f'Média: {media:.1f}')
        ax_box.axvline(mediana, color='green', linestyle='--', label=f'Mediana: {mediana:.1f}')
        ax_box.axvline(media + 3*std, color='orange', linestyle=':', label=f'+3σ: {(media + 3*std):.1f}')
        ax_box.axvline(media - 3*std, color='orange', linestyle=':', label=f'-3σ: {(media - 3*std):.1f}')

        ax_pdf.axvline(media, color='blue', linestyle='-', label=f'Média: {media:.1f}')
        ax_pdf.axvline(mediana, color='green', linestyle='--', label=f'Mediana: {mediana:.1f}')
        ax_pdf.axvline(media + 3*std, color='orange', linestyle=':', label=f'+3σ: {(media + 3*std):.1f}')
        ax_pdf.axvline(media - 3*std, color='orange', linestyle=':', label=f'-3σ: {(media - 3*std):.1f}')

    ax_pdf.set_xlim(xlim_lower, xlim_upper)
    ax_pdf.set_xlabel(column)
    ax_pdf.set_ylabel('Densidade')
    ax_pdf.set_title(f'PDF de {column}')
    ax_pdf.legend(fontsize=8, loc='upper left')

    plt.tight_layout(h_pad=2.5)
    plt.show()

def subplot_serie_hist(df, n_cols=3):
    """
    Plota múltiplas séries temporais em subplots com linhas de referência estatísticas.

    Parâmetros:
    -----------
    df : pandas.DataFrame
        DataFrame contendo os dados das séries temporais.
    n_cols : int, opcional
        Número de colunas no layout dos subplots. O padrão é 3.

    Descrição:
    ----------
    - Cria uma grade de subplots para visualizar múltiplas séries temporais simultaneamente.
    - Para cada variável, adiciona linhas de referência estatísticas (mínimo, máximo, limites do IQR).
    - Organiza os gráficos em uma grade de n_cols colunas, calculando automaticamente o número de linhas necessárias.
    - Adiciona títulos correspondentes aos nomes das colunas do DataFrame.

    Exemplo de uso:
    --------------
    subplot_serie_hist(df)  # Plota todas as colunas do DataFrame
    subplot_serie_hist(df[['pv_nafta', 'f_carg_nafta']], n_cols=2)  # Plota apenas as colunas selecionadas
    """

    n_vars = len(df.columns)
    n_rows = math.ceil(n_vars / n_cols)

    fig = make_subplots(rows=n_rows, cols=n_cols, subplot_titles=df.columns)

    for idx, col in enumerate(df.columns):
        row = idx // n_cols + 1
        col_idx = idx % n_cols + 1

        # Usa a lógica do serie_hist: plota a linha e adiciona linhas de referência (opcional)
        trace = px.line(df, y=col).data[0]
        fig.add_trace(trace, row=row, col=col_idx)

        # Adiciona linhas de referência (mínimo, máximo, IQR) igual ao serie_hist
        min_value = df[col].min()
        max_value = df[col].max()
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        loval = q1 - (1.5 * iqr)
        hival = q3 + (1.5 * iqr)

        fig.add_shape(type='line', x0=df.index.min(), y0=min_value, x1=df.index.max(), y1=min_value,
                    line=dict(color='gray', width=1, dash='dash'), row=row, col=col_idx)
        fig.add_shape(type='line', x0=df.index.min(), y0=max_value, x1=df.index.max(), y1=max_value,
                    line=dict(color='gray', width=1, dash='dash'), row=row, col=col_idx)
        fig.add_shape(type='line', x0=df.index.min(), y0=loval, x1=df.index.max(), y1=loval,
                    line=dict(color='orange', width=1, dash='dash'), row=row, col=col_idx)
        fig.add_shape(type='line', x0=df.index.min(), y0=hival, x1=df.index.max(), y1=hival,
                    line=dict(color='orange', width=1, dash='dash'), row=row, col=col_idx)

    # Adicionar uma legenda única para todas as linhas de referência
    fig.add_trace(
        go.Scatter(x=[None], y=[None], mode='lines', line=dict(color='gray', width=1, dash='dash'),
                  name='Min/Max', showlegend=True),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=[None], y=[None], mode='lines', line=dict(color='orange', width=1, dash='dash'),
                  name='IQR Limits (±1.5*IQR)', showlegend=True),
        row=1, col=1
    )

    fig.update_layout(
        height=250*n_rows,
        width=450*n_cols,
        showlegend=True,
        title_text="Séries Temporais das Variáveis",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig.show()

def _serialize_space(obj):
    # from skopt.space import Real, Integer, Categorical
    if isinstance(obj, Real):
        return {"_type": "Real", "low": obj.low, "high": obj.high, "prior": getattr(obj, "prior", None)}
    if isinstance(obj, Integer):
        return {"_type": "Integer", "low": obj.low, "high": obj.high, "prior": getattr(obj, "prior", None)}
    if isinstance(obj, Categorical):
        # categories pode ser array/numpy; transformar em lista simples
        return {"_type": "Categorical", "categories": list(obj.categories)}
    return obj

def _normalize_scalar(v):
    """Converte strings 'True'/'False' para bool e tenta interpretar tuplas/listas/números via ast.literal_eval."""
    # import ast
    if isinstance(v, str):
        s = v.strip()
        ls = s.lower()
        if ls == "true":
            return True
        if ls == "false":
            return False
        try:
            # converte "(50,)" -> (50,), "[50,50]" -> [50,50], "1e-3" -> float, "10" -> int
            return ast.literal_eval(s)
        except Exception:
            return v
    return v

def serialize_algo_configs(cfg):
    # from copy import deepcopy
    # from skopt.space import Real, Integer, Categorical
    out = deepcopy(cfg)
    for model_k, model_v in out.items():
        # Normalizar default_params (ex.: "True"/"(50,)" etc.)
        dp = model_v.get("default_params", {})
        if isinstance(dp, dict):
            model_v["default_params"] = {k: _normalize_scalar(v) for k, v in dp.items()}

        ss = model_v.get("search_space", {})
        if isinstance(ss, dict):
            for param_k, param_v in list(ss.items()):
                # se for um objeto skopt -> serializa
                if isinstance(param_v, (Real, Integer, Categorical)):
                    ss[param_k] = _serialize_space(param_v)
                    continue
                # se já for um dict serializável, normalizar categorias se for Categorical
                if isinstance(param_v, dict) and param_v.get("_type") == "Categorical":
                    cats = param_v.get("categories", [])
                    newcats = []
                    for c in cats:
                        newcats.append(_normalize_scalar(c))
                    param_v["categories"] = newcats
                    ss[param_k] = param_v
                    continue
                # Caso seja lista/tuple definido inline (ex.: [(50,), (100,)]), tentar normalizar conteúdo
                if isinstance(param_v, (list, tuple)):
                    # tenta detectar se elementos são strings representando tuplas/numeros
                    new_list = []
                    for elem in param_v:
                        new_list.append(_normalize_scalar(elem))
                    ss[param_k] = new_list
                    continue
                # fallback: mantém como está
                ss[param_k] = param_v
            model_v["search_space"] = ss
    return out

def _deserialize_space(obj):
    # from skopt.space import Real, Integer, Categorical
    # # já é um objeto skopt
    if isinstance(obj, (Real, Integer, Categorical)):
        return obj
    # se não for dict, retorna como está (fallback)
    if not isinstance(obj, dict):
        return obj

    def _make_hashable(x):
        if isinstance(x, list):
            return tuple(_make_hashable(v) for v in x)
        if isinstance(x, dict):
            return {k: _make_hashable(v) for k, v in x.items()}
        return x

    t = obj.get("_type")
    if t == "Real":
        return Real(obj["low"], obj["high"], prior=obj.get("prior"))
    if t == "Integer":
        return Integer(obj["low"], obj["high"], prior=obj.get("prior"))

    if t == "Categorical":
        cats = obj.get("categories", [])
        if not cats:
            raise ValueError("Categorical requires a non-empty 'categories' list")
        from ast import literal_eval
        norm_cats = []
        for c in cats:
            if isinstance(c, str):
                try:
                    parsed = literal_eval(c)
                    v = parsed
                except (ValueError, SyntaxError):
                    # preserve original string if it's not a literal encoding
                    v = c
            else:
                v = c
            # tornar cada categoria hashable (ex.: listas -> tuplas)
            v = _make_hashable(v)
            norm_cats.append(v)
        return Categorical(norm_cats)
    return obj

def compute_metrics(y_true, y_pred):
    return {
        "r2": r2_score(y_true, y_pred),
        "rmse": root_mean_squared_error(y_true, y_pred),
        "mse": mean_squared_error(y_true, y_pred),
        "mae": mean_absolute_error(y_true, y_pred)
    }

def get_regression_metric(abbreviated_name):
    """
    Retorna o nome completo da métrica de regressão baseado na abreviação

    Args:
        abbreviated_name (str): Nome abreviado da métrica

    Returns:
        str: Nome completo da métrica para usar no BayesSearchCV
    """
    # Dicionário de métricas de regressão para BayesSearchCV
    regression_metrics_dict = {
        'mse': 'neg_mean_squared_error',
        'rmse': 'neg_root_mean_squared_error',
        'mae': 'neg_mean_absolute_error',
        'medae': 'neg_median_absolute_error',
        'r2': 'r2',
        'var': 'explained_variance',
        'max_error': 'neg_max_error',
        'msle': 'neg_mean_squared_log_error',
        'rmsle': 'neg_root_mean_squared_log_error',
        'mape': 'neg_mean_absolute_percentage_error',
        'poisson': 'neg_mean_poisson_deviance',
        'gamma': 'neg_mean_gamma_deviance',
        'd2': 'd2_absolute_error_score'
    }
    return regression_metrics_dict.get(abbreviated_name.lower(), abbreviated_name)

def boxplot_algo(cv_df, test_df, metric_name, order):
    """
    Plota boxplot dos scores de validação cruzada (cv_df) e pontos de teste (test_df) para cada algoritmo,
    ordenando os algoritmos conforme a métrica (minimização ou maximização).
    A linha tracejada vermelha é traçada no menor valor (minimização) ou maior valor (maximização) do score de teste.

    Parâmetros:
    -----------
    cv_df : pd.DataFrame
        DataFrame com colunas ['Algoritmo', 'Score'] para os folds de validação cruzada.
    test_df : pd.DataFrame
        DataFrame com colunas ['Algoritmo', 'Score'] para os pontos de teste.
    metric_name : str
        Nome da métrica (ex: 'rmse', 'mae', 'r2', etc).
    """
    fig, ax = plt.subplots(figsize=(12, 6))

    # MINIMIZE_METRICS = {'rmse', 'mae', 'mse', 'msle', 'rmsle', 'mape', 'poisson', 'gamma', 'max_error', 'medae'}
    # MAXIMIZE_METRICS = {'r2', 'var', 'd2', 'explained_variance'}

    metric = metric_name.lower() if isinstance(metric_name, str) else str(metric_name).lower()

    sns.boxplot(
        x='Algoritmo',
        y='Score',
        data=cv_df,
        order=order,
        ax=ax,
        boxprops=dict(facecolor='darkblue', color='darkblue', alpha=0.3),
        medianprops=dict(color='darkblue'),
        whiskerprops=dict(color='darkblue'),
        capprops=dict(color='darkblue'),
        flierprops=dict(markerfacecolor='lightblue', markeredgecolor='darkblue')
        )

    sns.stripplot(
        x='Algoritmo',
        y='Score',
        data=test_df,
        order=order,
        color='red',
        ax=ax,
        jitter=True,
        size=6,
        zorder=10
        )

    # Decide se a linha horizontal é o menor ou maior valor, dependendo da métrica
    # Métricas onde menor é melhor (negativas no sklearn): 'rmse', 'mse', 'mae', 'medae', 'max_error', 'msle', 'rmsle', 'mape', 'poisson', 'gamma'
    # Métricas onde maior é melhor: 'r2', 'var', 'd2'
    metric_lower_is_better = MINIMIZE_METRICS
    metric = metric_name.lower() if isinstance(metric_name, str) else str(metric_name).lower()

    if not test_df.empty:
        if metric in metric_lower_is_better:
            test_score_line = test_df['Score'].min()
        else:
            test_score_line = test_df['Score'].max()
        ax.axhline(test_score_line, color='red', linestyle='--', linewidth=1.5, zorder=5)
    else:
        test_score_line = None

    cv_handle = Patch(facecolor='darkblue', edgecolor='darkblue', alpha=0.3, label='CV Folds')
    teste_handle = Line2D([0], [0], color='red', marker='o', linestyle='', markersize=5, label='Score do Teste')
    handles = [cv_handle, teste_handle]
    if test_score_line is not None:
        test_line_handle = Line2D([0], [0], color='red', linestyle='--', label='Linha Score Teste')
        handles.append(test_line_handle)

    ax.legend(handles=handles, loc='best')

    # Ajustes de visualização
    ax.set_title(f'Boxplot de Métricas ({METRIC_TO_OPT}) de Validação (CV) e Ponto de Teste Final')
    ax.set_xlabel('')
    ax.set_ylabel('Score')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

def boxplot_algo_grouped_by_scaler(df_box, metric_name, ncols=3, figsize_per_plot=(6,4)):
    """
    Plota subplots por scaler com a mesma escala Y.
    Usa df_box já preparado no notebook (colunas: Algoritmo, Scaler, Score, Type).
    Ordenação por scaler respeita METRIC_TO_OPT: 'r2' (maximizar) -> ordem decrescente,
    demais métricas consideradas de minimização -> ordem crescente.
    Além disso plota uma linha tracejada vermelha no melhor score de teste (global) passando por todos os subplots,
    e colore o(s) ponto(s) de teste correspondente(s) a esse melhor score em azul-marinho.
    Adiciona uma linha tracejada cinza indicando o valor do score do algoritmo baseline.
    """
    metric = metric_name.lower() if isinstance(metric_name, str) else str(metric_name).lower()

    cv_df = df_box[df_box['Type'] == 'CV']
    cv_df_merged = cv_df.copy()
    cv_df_merged['Algoritmo'] = cv_df_merged['Algoritmo'] + " (" + cv_df_merged['Scaler'] + ")"
    test_df = df_box[df_box['Type'] == 'Test']
    test_df_merged = test_df.copy()
    test_df_merged['Algoritmo'] = test_df_merged['Algoritmo'] + " (" + test_df_merged['Scaler'] + ")"

    if METRIC_TO_OPT.lower() in MINIMIZE_METRICS:
        ascending = True
    elif METRIC_TO_OPT.lower() in MAXIMIZE_METRICS:
        ascending = False
    else:
        # fallback: assume maximizar
        ascending = False

    # Atenção: para métricas de erro (minimização), o sklearn retorna valores negativos no cross-validation (cv_df)
    # enquanto os valores de teste (test_df) são positivos. Portanto, precisamos inverter o sinal dos scores do cv_df
    # para que ambos fiquem na mesma escala (positivos para erro, maiores = pior).

    if METRIC_TO_OPT.lower() in MINIMIZE_METRICS:
        cv_df_merged['Score'] = -cv_df_merged['Score']

    # ordenado pelo valor da métrica de teste
    order_teste = test_df_merged.groupby('Algoritmo')['Score'].mean().sort_values(ascending=ascending).index.tolist()
    order_cv = cv_df_merged.groupby('Algoritmo')['Score'].median().sort_values(ascending=ascending).index.tolist()

    df = df_box.copy()
    if metric in MINIMIZE_METRICS:
        df.loc[df['Type'] == 'CV', 'Score'] = -df.loc[df['Type'] == 'CV', 'Score']

    if metric in MAXIMIZE_METRICS:
        ascending = False
    else:
        ascending = True

    test_scores = df.loc[df['Type'] == 'Test', 'Score']
    global_best_test_score = None
    if not test_scores.empty:
        if metric in MINIMIZE_METRICS:
            global_best_test_score = test_scores.min()
        else:
            global_best_test_score = test_scores.max()

    # Score do baseline: primeiro algoritmo do primeiro scaler (assumindo ordenação)
    baseline_row = df[(df['Type'] == 'Test')].iloc[0] if not df[(df['Type'] == 'Test')].empty else None
    baseline_score = baseline_row['Score'] if baseline_row is not None else None

    scalers_list = sorted(df['Scaler'].unique(), key=lambda x: str(x))
    ymin = df['Score'].quantile(0.01)
    ymax = df['Score'].quantile(0.99)
    margin = 0.05 * (ymax - ymin) if ymax > ymin else 0.1
    y_min_plot = ymin - margin
    y_max_plot = ymax + margin

    n = len(scalers_list)
    nrows = math.ceil(n / ncols)
    fig_w = ncols * figsize_per_plot[0]
    fig_h = nrows * figsize_per_plot[1]
    fig, axes = plt.subplots(nrows, ncols, figsize=(fig_w, fig_h), squeeze=False)
    axes = axes.flatten()

    tol = 1e-8

    # Inverter o gradiente de cores do boxplot (usar 'Blues_r')
    box_palette = 'Blues_r'

    for idx, scaler in enumerate(scalers_list):
        ax = axes[idx]
        df_cv_sub = df[(df['Scaler'] == scaler) & (df['Type'] == 'CV')]
        df_test_sub = df[(df['Scaler'] == scaler) & (df['Type'] == 'Test')]

        if df_cv_sub.empty and df_test_sub.empty:
            ax.text(0.5, 0.5, 'Sem dados', ha='center', va='center')
            ax.set_title(f'Scaler: {scaler}')
            ax.set_ylim(y_min_plot, y_max_plot)
            ax.set_xticks([])
            continue

        algs_cv = set(df_cv_sub['Algoritmo'].unique())
        algs_test = set(df_test_sub['Algoritmo'].unique())
        algs_present = list(algs_cv.union(algs_test))

        order_for_scaler = []
        if not df_test_sub.empty:
            order_for_scaler = df_test_sub.groupby('Algoritmo')['Score'].mean().sort_values(ascending=ascending).index.tolist()
        elif not df_cv_sub.empty:
            order_for_scaler = df_cv_sub.groupby('Algoritmo')['Score'].median().sort_values(ascending=ascending).index.tolist()
        order_for_scaler = [a for a in order_for_scaler if a in algs_present]
        if not order_for_scaler:
            order_for_scaler = sorted(algs_present, key=lambda x: str(x))

        sns.boxplot(
            x='Algoritmo',
            y='Score',
            data=df_cv_sub,
            order=order_for_scaler,
            ax=ax,
            palette=box_palette,
            boxprops=dict(alpha=0.6)
        )

        if not df_test_sub.empty:
            sns.stripplot(
                x='Algoritmo',
                y='Score',
                data=df_test_sub,
                order=order_for_scaler,
                ax=ax,
                color='red',
                size=7,
                jitter=False,
                marker='D'
            )
            if global_best_test_score is not None:
                matches = df_test_sub[np.isclose(df_test_sub['Score'].astype(float), float(global_best_test_score), atol=tol)]
                if not matches.empty:
                    for _, row in matches.iterrows():
                        alg = row['Algoritmo']
                        if alg not in order_for_scaler:
                            continue
                        x_pos = order_for_scaler.index(alg)
                        y_val = row['Score']
                        ax.scatter(x_pos, y_val, color='navy', edgecolor='white', s=49, marker='D', zorder=30)

        positions = np.arange(len(order_for_scaler))
        ax.set_xticks(positions)
        ax.set_xticklabels(order_for_scaler, rotation=45, ha='right')

        ax.set_title(f'Scaler: {scaler}')
        ax.set_ylim(y_min_plot, y_max_plot)
        ax.set_xlabel('')
        if idx % ncols == 0:
            ax.set_ylabel(metric_name)
        else:
            ax.set_ylabel('')
        ax.tick_params(axis='x')

        # Linha tracejada vermelha do melhor score de teste global
        if global_best_test_score is not None:
            ax.axhline(global_best_test_score, color='red', linestyle='--', linewidth=1.5, zorder=20, label='Melhor Score Teste (global)')
        # Linha tracejada cinza do baseline
        if baseline_score is not None:
            ax.axhline(baseline_score, color='gray', linestyle=':', linewidth=1.5, zorder=10, label='Baseline')

    for j in range(n, len(axes)):
        fig.delaxes(axes[j])

    handles = [
        Patch(facecolor='darkblue', edgecolor='k', alpha=0.6, label='CV folds'),
        Line2D([0], [0], color='red', marker='D', linestyle='', markersize=7, label='Test score'),
    ]
    if global_best_test_score is not None:
        handles.append(Line2D([0], [0], color='red', linestyle='--', label='Melhor Score Teste (global)'))
    if baseline_score is not None:
        handles.append(Line2D([0], [0], color='gray', linestyle=':', label='Baseline'))

    # Reposicionar a legenda para fora do gráfico, acima do título
    fig.legend(handles=handles, loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.08))
    plt.suptitle(f'Comparativo por Scaler (Métrica: {metric_name})', fontsize=14, y=1.03)
    plt.tight_layout(rect=[0, 0, 1, 0.98])
    plt.show()

    # Tabela resumo dos resultados ordenada pela métrica de teste (melhor desempenho primeiro)
    summary_table = test_df_merged.copy()
    summary_table['Algoritmo'] = summary_table['Algoritmo'].str.replace(r"\s*\(.*\)", "", regex=True)
    summary_table = summary_table.sort_values('Score', ascending=ascending).reset_index(drop=True)
    summary_table['Ordem'] = summary_table.index + 1

    print(f"{'Ordem':<6} {'Algoritmo':<25} {'Scaler':<10} {'Score':<10} {'Métrica':<10}")
    print("-" * 65)
    for idx, row in summary_table.iterrows():
        # Coloca em negrito o melhor resultado (primeira linha)
        if idx == 0:
            print(f"\033[1m{row['Ordem']:<6} {row['Algoritmo']:<25} {row['Scaler']:<10} {row['Score']:<10.4f} {METRIC_TO_OPT:<10}\033[0m")
        else:
            print(f"{row['Ordem']:<6} {row['Algoritmo']:<25} {row['Scaler']:<10} {row['Score']:<10.4f} {METRIC_TO_OPT:<10}")

## 2.4 Configuração dos algoritmo

### 2.4.1 Algoritmos de Machine Learning

In [ ]:
# Configuração dos metadados dos algoritmos (JSON)
ALGO_CONFIGS = {
    "linear": {
        "alias": "Linear",
        "model_class": "LinearRegression",
        "module": "sklearn.linear_model",
        "default_params": {},
        "search_space": {
            'positive': Categorical(["False", "True"])
        },
        "default_metric": "r2"
    },
    "ridge": {
        "alias": "Ridge",
        "model_class": "Ridge",
        "module": "sklearn.linear_model",
        "default_params": {"random_state": 42},
        "search_space": {
            "alpha": Real(1e-3, 10.0, prior="log-uniform")
        },
        "default_metric": "r2"
    },
    "lasso": {
        "alias": "Lasso",
        "model_class": "Lasso",
        "module": "sklearn.linear_model",
        "default_params": {"random_state": 42, "max_iter": 2000},
        "search_space": {
            "alpha": Real(1e-3, 10.0, prior="log-uniform")
        },
        "default_metric": "r2"
    },
    "elastic_net": {
        "alias": "ElasticNet",
        "model_class": "ElasticNet",
        "module": "sklearn.linear_model",
        "default_params": {"random_state": 42, "max_iter": 2000},
        "search_space": {
            "alpha": Real(1e-3, 10.0, prior="log-uniform"),
            "l1_ratio": Real(0.0, 1.0, prior="uniform")
        },
        "default_metric": "r2"
    },
    "svr": {
        "alias": "SVR",
        "model_class": "SVR",
        "module": "sklearn.svm",
        "default_params": {},
        "search_space": {
            "C": Real(0.1, 100.0, prior="log-uniform"),
            "gamma": Real(1e-4, 1e-1, prior="log-uniform"),
            "kernel": Categorical(['rbf', 'linear', 'poly'])
        },
        "default_metric": "r2"
    },
    "knn": {
        "alias": "KNN",
        "model_class": "KNeighborsRegressor",
        "module": "sklearn.neighbors",
        "default_params": {"n_jobs": -1},
        "search_space": {
            "n_neighbors": Integer(3, 20),
            "weights": Categorical(['uniform', 'distance'])
        },
        "default_metric": "r2"
    },
    "decision_tree": {
        "alias": "DecisionTree",
        "model_class": "DecisionTreeRegressor",
        "module": "sklearn.tree",
        "default_params": {"random_state": 42},
        "search_space": {
            "max_depth": Integer(3, 20),
            "min_samples_split": Integer(2, 20),
            "min_samples_leaf": Integer(1, 10),
            "max_features": Categorical(['sqrt', 'log2'])
        },
        "default_metric": "r2"
    },
    "random_forest": {
        "alias": "RandomForest",
        "model_class": "RandomForestRegressor",
        "module": "sklearn.ensemble",
        "default_params": {"random_state": 42, "n_jobs": -1},
        "search_space": {
            "n_estimators": Integer(50, 300),
            "max_depth": Integer(5, 20),
            "min_samples_split": Integer(2, 10),
            "min_samples_leaf": Integer(1, 5),
            "max_features": Categorical(['sqrt', 'log2'])
        },
        "default_metric": "r2"
    },
    "ada_boost": {
        "alias": "AdaBoost",
        "model_class": "AdaBoostRegressor",
        "module": "sklearn.ensemble",
        "default_params": {"random_state": 42},
        "search_space": {
            "n_estimators": Integer(50, 200),
            "learning_rate": Real(0.01, 2.0, prior="log-uniform"),
            "loss": Categorical(['linear', 'square', 'exponential'])
        },
        "default_metric": "r2"
    },
    "gradient_boosting": {
        "alias": "GradientBoost",
        "model_class": "GradientBoostingRegressor",
        "module": "sklearn.ensemble",
        "default_params": {"random_state": 42},
        "search_space": {
            "n_estimators": Integer(50, 200),
            "max_depth": Integer(3, 10),
            "learning_rate": Real(0.01, 0.3, prior="log-uniform"),
            "subsample": Real(0.6, 1.0)
        },
        "default_metric": "r2"
    },
    "bagging": {
        "alias": "Bagging",
        "model_class": "BaggingRegressor",
        "module": "sklearn.ensemble",
        "default_params": {"random_state": 42, "n_jobs": -1},
        "search_space": {
            "n_estimators": Integer(10, 100),
            "max_samples": Real(0.5, 1.0),
            "max_features": Real(0.5, 1.0)
        },
        "default_metric": "r2"
    },
    "extra_trees": {
        "alias": "ExtraTrees",
        "model_class": "ExtraTreesRegressor",
        "module": "sklearn.ensemble",
        "default_params": {"random_state": 42, "n_jobs": -1},
        "search_space": {
            "max_depth": Integer(5, 20),
            "min_samples_split": Integer(2, 10),
            "min_samples_leaf": Integer(1, 5)
        },
        "default_metric": "r2"
    },
    "lightgbm": {
        "alias": "LightGBM",
        "model_class": "LGBMRegressor",
        "module": "lightgbm",
        "default_params": {
            "random_state": 42,
            "verbose": -1
        },
        "search_space": {
            "n_estimators": {
                "_type": "Integer",
                "low": 50,
                "high": 300,
                "prior": "uniform"
            },
            "max_depth": {
                "_type": "Integer",
                "low": 3,
                "high": 10,
                "prior": "uniform"
            },
            "learning_rate": {
                "_type": "Real",
                "low": 0.01,
                "high": 0.3,
                "prior": "log-uniform"
            },
            "subsample": {
                "_type": "Real",
                "low": 0.6,
                "high": 1.0,
                "prior": "uniform"
            },
            "colsample_bytree": {
                "_type": "Real",
                "low": 0.6,
                "high": 1.0,
                "prior": "uniform"
            },
            "num_leaves": {
                "_type": "Integer",
                "low": 20,
                "high": 150,
                "prior": "uniform"
            },
            "min_child_samples": {
                "_type": "Integer",
                "low": 5,
                "high": 100,
                "prior": "uniform"
            },
        },
        "default_metric": "r2"
    },
    "xgboost": {
        "alias": "XGBoost",
        "model_class": "XGBRegressor",
        "module": "xgboost",
        "default_params": {"random_state": 42, "verbosity": 0},
        "search_space": {
            "n_estimators": Integer(50, 300),
            "max_depth": Integer(3, 10),
            "learning_rate": Real(0.01, 0.3, prior="log-uniform"),
            "subsample": Real(0.6, 1.0),
            "colsample_bytree": Real(0.6, 1.0)
        },
        "default_metric": "r2"
    },
    "catboost": {
        "alias": "CatBoost",
        "model_class": "CatBoostRegressor",
        "module": "catboost",
        "default_params": {
            "random_seed": 42,
            "verbose": 0,
            "allow_writing_files": "False"
        },
        "search_space": {
            "iterations": {
                "_type": "Integer",
                "low": 50,
                "high": 300,
                "prior": "uniform"
            },
            "depth": {
                "_type": "Integer",
                "low": 3,
                "high": 10,
                "prior": "uniform"
            },
            "learning_rate": {
                "_type": "Real",
                "low": 0.01,
                "high": 0.3,
                "prior": "log-uniform"
            },
            "subsample": {
                "_type": "Real",
                "low": 0.6,
                "high": 1.0,
                "prior": "uniform"
            },
            "colsample_bylevel": {
                "_type": "Real",
                "low": 0.6,
                "high": 1.0,
                "prior": "uniform"
            },
            "l2_leaf_reg": {
                "_type": "Real",
                "low": 1.0,
                "high": 10.0,
                "prior": "uniform"
            }
        },
        "default_metric": "r2"
    },
    "mlp": {
        "alias": "MLP",
        "model_class": "MLPRegressor",
        "module": "sklearn.neural_network",
        "default_params": {
            "random_state": 42,
            "max_iter": 1000,
            "early_stopping": "True",
            "n_iter_no_change": 10,
            "tol": 1e-4,
            "solver": "adam",
            "learning_rate_init": "1e-3",
            "batch_size": "auto"

        },
        "search_space": {
            "hidden_layer_sizes": {
                "_type": "Categorical",
                "categories": [16, 32, 64, 128, 256, 512]
            },
            "activation": {
                "_type": "Categorical",
                "categories": [
                    "relu",
                    "tanh",
                    "logistic"
                ]
            },
            "alpha": {
                "_type": "Real",
                "low": 1e-5,
                "high": 1.0,
                "prior": "log-uniform"
            },
            "learning_rate": {
                "_type": "Categorical",
                "categories": [
                    "constant",
                    "adaptive",
                    "invscaling"
                ]
            },
        },
        "default_metric": "r2"
    }
}

ALGO_CONFIGS_SERIALIZABLE = serialize_algo_configs(ALGO_CONFIGS)

with open('algo_configs.json', 'w', encoding='utf-8') as f:
    json.dump(ALGO_CONFIGS_SERIALIZABLE, f, indent=4, ensure_ascii=False)

### 2.4.2 Algoritmos de Padronização

In [ ]:
# Métodos de padronização
SCALERS_CONFIGS = [
    ("Minmax", MinMaxScaler()),
    ("Standard", StandardScaler()),
    ("YeoJohn", PowerTransformer(method='yeo-johnson')), # Box-Cox requer dados positivos
    # # # Vamos deixar comentado para eventual uso em outros trabalhos.
    # ("Maxabs", MaxAbsScaler()),
    # ("Robust", RobustScaler()),
    # ("Normal", Normalizer())
]

# 3. Dados: entendimento, carga e qualidade

O dataset **Pressão de Vapor da nafta** é um conjunto de dados previamente tratado na SPRINT de Análise de Dados e Boas Práticas.

Esse dataset já passou por vários tratamentos, dentre eles os de valores nulos, faltantes e de outliers.

Será necessária uma breve análise exploratória dos dados de forma a torná-lo uma fonte de dados ainda mais curada para o uso.

## 3.1. Atributos do dataset

O dataset Pressão de Vapor de nafta pode ser assim resumido:

***pv_nafta***: pressão de vapor da nafta (unidade de pressão)<br>
***t_carg_nafta***: temperatura da carga da fracionadora de nafta (unidade de temperatura)<br>
***t_fund_nafta***: temperatura do fundo da fracionadora de nafta (unidade de temperatura)<br>
***t_aque_nafta***: temperatura do aquecedor de fundo da fracionadora de nafta (unidade de temperatura)<br>
***t_esup_nafta***: temperatura estágio superior interno da fracionadora de nafta (unidade de temperatura)<br>
***t_eint_nafta***: temperatura estágio intermediário interno da fracionadora de nafta (unidade de temperatura)<br>
***t_einf_nafta***: temperatura estágio inferior interno da fracionadora de nafta (unidade de temperatura)<br>
***t_topo_nafta***: temperatura de topo da fracionadora de nafta (unidade de temperatura)<br>
***t_lhtp_nafta***: temperatura da linha de topo da fracionadora de nafta (unidade de temperatura)<br>
***p_topo_nafta***: pressão de topo da fracionadora de nafta (unidade de pressão)<br>
***f_carg_nafta***: vazão de carga da fracionadora de nafta (volume por unidade de tempo)<br>
***f_refl_nafta***: vazão de refluxo de topo da fracionadora de nafta (volume por unidade de tempo)<br>

## 3.2. Carregamento do dataset

In [ ]:
url = "https://raw.githubusercontent.com/fdamata/pucrj-machinelearning-mvp-ml/main/dataset_pv_nafta_ml.xlsx"
# url = 'dataset_pv_nafta_ml.xlsx'


df = pd.read_excel(url, engine='openpyxl') # Carregamento do dataset
display(df.head()) # Imprime as primeiras linhas do dataset

## 3.3. Análise de dados

Nesta etapa buscamos entender a distribuição, as relações e as características das variáveis, o que é crucial para as etapas subsequentes.

### 3.3.1. Total e tipo dos atributos

O dataset possui 906 observações. Os doze (12) atributos são de tipo numérico (float).

In [ ]:
print(f"Total de instâncias: {len(df)}")

tags = df.columns.to_list()

# Definição de que a variável dependente está na primeira coluna
target = tags[0]
inputs = [tag for tag in tags if tag != target]

print(f'Num. de atributos: {len(tags)}')
# Separação entre variável dependente (target) e independentes (inputs)
print(f'target = {target}')
print(f'inputs = {inputs}')
# print(f'Num. inputs = {len(inputs)}')

print("\nTipos de dados por coluna:\n")

print(df.info())

### 3.3.2. Análise de comportamento temporal

O objetivo é observar o comportamento das variáveis ao longo do tempo.

In [ ]:
subplot_serie_hist(df, n_cols=3)

É possível observar oscilações atípicas, mas nesse contexto podem voltar a repetir. Entendemos ser necessário tentar capturar esse tipo de contribuição na previsão da propriedade.

### 3.3.3. Estatísticas Descritivas

Estatísticas descritivas fornecem um resumo das características numéricas, incluindo média, desvio padrão, mínimo, máximo e quartis.

In [ ]:
# estasticas descritivas básicas do dataset
print('Estatísticas descritivas:\n')
# Exibe as estatísticas descritivas transpostas com floats formatados para 2 casas decimais
with pd.option_context('display.float_format', '{:.2f}'.format):
    print(df.describe().T)
plot_boxplot_pdf(df, n_cols=4)

É possível observar que nas unidades de engenharia utilizadas não há um único padrão para todas as distribuições.<br>
Há algumas distribuições que aparentam a normalidade, mas que requerem avaliação específica para confirmação das hipóteses.

In [ ]:
# Loop para aplicar a função teste_n em todas as colunas do DataFrame
print("Teste de normalidade para todas as variáveis:")
print("=" * 60)

normality_results = {}
for col in df.columns:
    print(f"Variável: '{col}'")
    _, p_valor = teste_n(df, col)
    normality_results[col] = "Normal" if p_valor > 0.05 else "Não normal"
    print("-" * 60)

# # Criar um DataFrame com os resultados para melhor visualização
# normality_df = pd.DataFrame.from_dict(normality_results, orient='index', columns=['Distribuição'])
# print("\nResumo dos testes de normalidade:")
# print(normality_df)

# Calcular a proporção de variáveis com distribuição normal
normal_count = sum(1 for status in normality_results.values() if status == "Normal")
print(f"\nProporção de variáveis com distribuição normal: {normal_count}/{len(df.columns)} ({normal_count/len(df.columns)*100:.1f}%)")

Nesse dataset 50% dos atributos apresentam distribuição normal.

### 3.3.4. Matriz de Correlação

A matriz de correlação mede a força e a direção de uma relação linear entre os atributos do dataset.<br>
Valores próximos a 1 indicam uma forte correlação positiva, -1 uma forte correlação negativa, e 0 ausência de correlação linear.

Para simplificar a visualização de correlação linear, o mapa de calor será realizado com o valor absoluto da correlação [0,1] ao invés de [-1, 1]

In [ ]:
# Matriz de correlação
print("\nPara simplificar a visualização de correlação linear, o mapa de calor será realizado com o valor absoluto da correlação [0,1] ao invés de [-1, 1]\n")
# print(df.corr())
calcula_corr(df)
pairplot_corr_hm(df, figsize=(24, 24), hist_bins=30, s=10, alpha=0.5)

### 3.3.5. Análise de VIF

Análise de VIF (Variance Inflation Factor ou Fator de Inflação da Variância) é uma medida para quantificar o grau de multicolinearidade entre as variáveis independentes do dataset. A multicolinearidade ocorre quando duas ou mais variáveis independentes estão altamente correlacionadas, o que pode dificultar a estimativa dos coeficientes da regressão e afetar a interpretação dos resultados, principalmente para modelos lineares.

Interpretação do VIF:
- VIF = 1: não há correlação entre a variável $i$ e as outras variáveis. A variância não está inflacionada.
- 1 < VIF < 5: a correlação é moderada, mas geralmente não é uma preocupação.
- VIF ≥ 5: indica uma alta correlação e pode ser motivo de preocupação; a variável pode estar contribuindo para a multicolinearidade.
- VIF ≥ 10: geralmente considerado um sinal forte de multicolinearidade, e pode ser necessário considerar a remoção da variável ou a aplicação de técnicas de regularização.

In [ ]:
calcula_vif(df,target)

Processo iterativo de remoção de variáveis independentes até a obtenção de maior VIF < 10 (desejavelmente < 5).<br>
Nesse momento é importante analisar o VIF não apenas como um critério estatístico, mas também com o conhecimento de domínio.<br>

Pelo conhecimento de domínio as variáveis independentes **'t_topo_nafta'**, **'t_esup_nafta'**, **'t_eint_nafta'** e **'t_lhtp_nafta'** apresentam interdependência por representar parte do equilíbrio termodinâmico nessa seção da torre. Como também observado nas correlações, podemos aplicar redução de dimensionalidade do dataset pela supressão de alguma delas.

Nesse conjunto de variáveis observamos que as 4 variáveis estão entre as 6 maiores contribuições de VIF. O objetivo é realizar a menor supressão de regressores que carreguem informações semelhantes. Das 4 variáveis **'t_eint_nafta'** é a que carrega menor VIF, indicando ser a candidata a permanecer, pois ao suprimir uma das variáveis que encontram-se com VIF maior, provavelmente se enquadrará em VIF < 5.

Iniciando a exclusão da variável de maior VIF (**'t_topo_nafta**) já tivemos uma melhora significativa, no entanto ainda com VIF para as variáveis desse grupo superior a 10. Então foi excluída mais uma variável (**'t_esup_nafta'**) alcançando então VIF < 5 para as variáveis desse grupo.

In [ ]:
# df1 = df.drop(columns=['t_lhtp_nafta','t_topo_nafta', 't_esup_nafta','t_eint_nafta' ])
df1 = df.drop(columns=['t_topo_nafta','t_esup_nafta'])
calcula_vif(df1,target)
# calcula_corr(df1)
# pairplot_corr_hm(df1, figsize=(24, 24), hist_bins=30, s=10, alpha=0.5)

De forma análoga à seção anterior, o conjunto de **'t_fund_nafta'**, **'t_aque_nafta'**, **'t_carga_nafta'** e **'t_einf_nafta'** representam parte do equilíbrio termodinâmico na região inferior da torre, permitindo-nos realizar a redução de dimensionalidade como realizado anteriormente.

As candidatas naturais pelo critério estatístico seriam **'t_fund_nafta'** ou **'t_aque_nafta'**, no entanto a **'t_fund_nafta'** melhor caracteriza o que poderia ser um indicativo de preservação ou supressão de leves na nafta, pois já está saindo da torre para produção do produto final, entanto **'t_aque_nafta'** é um reciclo que não necessariamente indica o acabamento do produto, mas sim uma oportunidade para alteração do equilíbrio termodinâmica na torre, mas que está sendo contempladas pelas demais variáveis dessa mesma seção que serão preservdas.

Com isso, excluindo **'t_aque_nafta'** enquadramos o critérios da VIF e preservando melhor as informações do processo físico-químico.

In [ ]:
# df2 = df1.drop(columns=['t_fund_nafta','t_aque_nafta','t_carg_nafta','t_einf_nafta'])
df2 = df1.drop(columns=['t_aque_nafta'])
calcula_vif(df2,target)
# calcula_corr(df2)
# pairplot_corr_hm(df2, figsize=(24, 24), hist_bins=30, s=10, alpha=0.5)

In [ ]:
# Resumo das distribuições dos atributos remanescentes

plot_boxplot_pdf(df2)

### 3.3.6. Criação de novas características



Razão de refluxo: uma variável derivada, definida como a relação entre a vazão de refluxo de topo **'f_refl_nafta'** e a vazão de carga da fracionadora **'f_carg_nafta'** (**'f_refl_nafta'**/**'f_carg_nafta'**). Esta variável representa um parâmetros operacionais que pode ser críticos em fracionadoras, pois determina a eficiência da separação dos componentes.

Eficiência da separação: valores mais altos de razão de refluxo proporcionam maior contato entre as fases líquida e vapor em cada estágio do fracionamento, resultando em melhor separação dos componentes com diferentes volatilidades.

Controle da composição do produto: ao aumentar a razão de refluxo, mais componentes leves são retidos e retornam à fracionadora em vez de saírem no produto de topo, esperando-se uma elevação da pressão de vapor da nafta que sai pelo fundo da fracionadora.

Estabilidade operacional: uma razão de refluxo adequada ajuda a manter condições operacionais estáveis, reduzindo flutuações na qualidade do produto.

Perfil térmico da coluna: influencia diretamente o gradiente de temperatura ao longo da fracionadora, afetando a distribuição dos componentes em cada estágio.

Implicações: na prática, existe um trade-off importante no ajuste da razão de refluxo:

- Razão de refluxo baixa: menor consumo energético e maior capacidade de processamento, porém com qualidade de separação potencialmente comprometida.

- Razão de refluxo alta: melhor separação e controle mais preciso da pressão de vapor do produto, porém com maior consumo energético e menor capacidade de processamento.

Para a predição da pressão de vapor da nafta, a razão de refluxo representa uma variável derivada de alto valor preditivo, pois sintetiza em um único parâmetro a interação entre duas variáveis operacionais críticas que afetam diretamente o equilíbrio termodinâmico e a composição do produto final.

A inclusão desta variável no modelo preditivo provavelmente aumentará seu poder explicativo, capturando um mecanismo de controle operacional que impacta a propriedade alvo que estamos tentando prever.

In [ ]:
df2['r_refl_nafta'] = df2['f_refl_nafta'] / df2['f_carg_nafta']
plot_boxplot_pdf_indiv(df2, 'f_refl_nafta')

O novo atributo criado apresenta distribuição assimétrica à direita, com uma cauda da distribuição se estendendo nesse sentido, indicando a presença de valores mais altos menos frequentes. Esse comportamento é derivado da variável **'f_refl_nafta'** que deu origem a essa nova criação, conforme já apresentado anteriormente.

Vamos avaliar como ficaram as correlações e a VIF após essa nova vaiável.

In [ ]:
calcula_vif(df2,target)
calcula_corr(df2)
# pairplot_corr_hm(df2, figsize=(24, 24), hist_bins=30, s=10, alpha=0.5)

É possível observar uma elevada correlação, conforme esperado, entre **'f_refl_nafta'** e a nova **'r_refl_nafta'**. Em função da relevância dessa nova variável, conforme explicado anteriormente e até mesmo da discreta maior correlação com a variável alvo, vamos suprimir a **'f_refl_nafta'** para reduzir a VIF (reduzir a mulcolinearidade).

In [ ]:
df3 = df2.drop(columns=['f_refl_nafta'])
calcula_vif(df3,target)

Como a VIF ficou novamente abaixo de 5 vamos parar a remoção de variáveis nessa etapa.

# 4. Separação e divisão do dataset entre features (X) e target (y)

- Iniciamos aqui as definições paramétricas para o problema de aprendizado de máquina. Definimos uma semente (SEED) para garantir a reprodutibilidade. Os demais parâmetros estão relacionados aos critérios de separação dos dados entre treino e teste (SPLIT), a quantidade de 'folds" na validação cruzada (CV_FOLDS) e por último, e não menos importante, a definição da métrica de avaliação.

A escolha pelo R² é muito mais pela intruição que essa métrica trás referente à explicação de quanto da variância está sendo explicada pelo modelo, mas poderia também termos escolhidos a RMSE por tra´s também uma boa intuição, já que representa o erro absoluto médio na escala de engenharia do target.

In [ ]:
X = df3.drop(target, axis=1)
y = df3[target].copy()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=SPLIT, random_state=SEED, stratify=y if PROBLEM_TYPE=="classificacao" else None
)
print("Treino:", X_train.shape, "| Teste:", X_test.shape)
print("Tipo:", PROBLEM_TYPE)
print("Target:", target)
print("Nº features:", X.shape[1])

Separação prévia entre dados de treino e de teste para evitar vazamento de dados. Os dados de teste são considerados intocados.

# 5 Seleção de algorítmos e definição da Baseline

## 5.1 Algorítmos

Selecionados previamente no início do notebook

### 5.1.1 Algorítmos de regressão

Algoritmos Lineares: Assumem relação linear entre features e target, são interpretáveis, mas limitados em capturar relações complexas.
  - Linear, Ridge, Lasso, ElasticNet, SVR (kernel linear)

Algoritmos Baseados em Distância: Fazem previsões com base na similaridade/distância entre pontos de dados, não fazem suposições sobre a distribuição.
  - KNN, SVR (kernels não lineares)

Algoritmos Baseados em Árvores:
   - Árvore Única: Divide o espaço de features recursivamente, altamente interpretável mas propensa a overfitting.
     - DecisionTree
   - Ensembles - Bagging: Reduz a variância combinando múltiplos modelos treinados em diferentes subconjuntos dos dados.
     - RandomForest, Bagging, ExtraTrees
   - Ensembles - Boosting: Reduz o viés construindo modelos sequencialmente, cada um focando nos erros dos anteriores.
     - AdaBoost, GradientBoost, XGBoost, LightGBM, CatBoost

### 5.1.2 Métodos de padronização (Scalers)

Considerando a estratégia de escolha dos algorítmos de regressão e sabendo que algoritmos baseados em árvores geralmente não são sensíveis à escala das features (talvez influenciados por grande assimetria), devemos nos concentrar na escolha dos scalers para os algoritmos lineares e baseados em distância. E ainda, como é sabido que outliers foram tratados na etapa de pre=processamento, RobustScaler pode ser preterido.

Desta forma, vamos selecionar StandardScaler, MinMaxScaler e para avaliar eventuais impactos de assimetria na distribuição de algumas features, vamos considerar ainda uma PowerTransformer com o método de 'Yeo-Johnson', dado que esse métrodo é mais robusto que o 'Box-Cox' por contemplar tratamento de valores negativos.

### 5.1.3 Métricas calculadas

Em relação às métricas a escolha foi baseada na representatividade da explicação da variabilidade (R²) pelo modelo, na dimensão do erro na escala de engenharia da medida do target (RMSE), o erro quadrático médio (MSE), aonde valores mais extremos de erro tem maior penalização e também o erro absoluto (MAE).


In [ ]:
# Carregando dos algoritmos de regressão
filename = 'algo_configs.json'
if os.path.getsize(filename) == 0:
    raise ValueError(f"O arquivo '{filename}' está vazio. Gere ou preencha o arquivo antes de carregar.")
with open(filename, 'r', encoding='utf-8') as f:
    algo_configs = json.load(f)

# Métodos de padronização
scalers = SCALERS_CONFIGS

# Definição das métricas de avaliação
scoring = {
    'r2': 'r2',
    'rmse': 'neg_root_mean_squared_error',
    'mse': 'neg_mean_squared_error',
    'mae': 'neg_mean_absolute_error'
}

# Sumário das configurações carregadas
print("Algorítmos considerados:\n")
for i, (scaler_name,scaler) in enumerate(scalers):
    print(f"Scaler[{i}]: {scaler_name}")

print('')

for i, (key, cfg) in enumerate(algo_configs.items()):
    alias = cfg.get('alias')
    model_class = cfg.get('model_class')
    module_name = cfg.get('module')
    default_params = cfg.get('default_params')
    search_space = cfg.get('search_space')
    print(f"Algoritmos[{i}]: {alias}")

## 5.2 Baseline

Como Baseline a escolha foi pelo modelo de regressão linear (OLS) em função da sua simplicidade.

In [ ]:
# Modelo baseline
baseline = algo_configs[list(algo_configs.keys())[0]]
baseline_algorithm = baseline['model_class']
baseline_model = baseline['alias']
print(f"\nBaseline - Modelo: {baseline_model}")
print(f"Baseline - Algoritmo: {baseline_algorithm}\n")

In [ ]:
# --- Célula de Otimização de Hiperparâmetros com BayesSearchCV ---
N_ITER = 50  # Número de iterações da busca. Aumentar para uma busca mais exaustiva.

results = {}
cv_results = {}
test_results = {}
opt_res = {}
opt_res2 = {}

# Criar o objeto de cross-validation explicitamente
cv_splitter = KFold(n_splits=CV_FOLDS, shuffle=True, random_state=SEED)

# Criar um callback DeltaYStopper com delta pequeno para demonstração
delta_metric = 0.0001 if METRIC_TO_OPT in ['r2'] else 0.01

for scaler_name, scaler in scalers:
    for i, (algo_name, config) in enumerate(algo_configs.items()):
        print(f"Executando HPO para: {config.get('alias')} - {scaler_name}")
        start_time = time.time()

        model_class = config["model_class"]
        module_name = config["module"]
        model_cls = getattr(importlib.import_module(module_name), model_class)
        model = model_cls(**config["default_params"])

        pipe = Pipeline([
            ("scaler", scaler),
            ("model", model)
        ])

        # Reconstruir search_space com o prefixo 'model__' para uso com Pipeline
        search_space = {f"model__{k}": _deserialize_space(v) for k, v in config.get("search_space", {}).items()}

        opt = BayesSearchCV(
            estimator=pipe,
            search_spaces=search_space,
            n_iter=N_ITER,
            cv=cv_splitter,
            scoring=get_regression_metric(METRIC_TO_OPT),
            random_state=SEED,
            n_jobs=-1,
            return_train_score=True,
            verbose=0
        )

        opt.fit(X_train, y_train, callback=[DeltaYStopper(delta=delta_metric, n_best=15)])

        end_time = time.time()
        elapsed = end_time - start_time
        print(f" - Tempo estimado para {config.get('alias')} - {scaler_name}: {elapsed:.2f} segundos")

        best_model = opt.best_estimator_
        best_idx = opt.best_index_

        # Histórico completo da otimização bayesiana para cada (algoritmo, scaler)
        opt_res[algo_name, scaler_name] = {
            "optimizer_result": opt.optimizer_results_[0],
            "elapsed_time": elapsed
        }

        y_pred_train = best_model.predict(X_train)
        y_pred_test = best_model.predict(X_test)

        # Resultados finais de cada combinação de algoritmo e scaler:
        # Melhores hiperparâmetros, métricas (treino, teste, CV) e
        # os scores dos folds apenas para o melhor modelo de cada busca.
        results[algo_name,scaler_name] = {
            "params": opt.best_params_,
            "scores_dev": compute_metrics(y_train, y_pred_train),
            "scores_test": compute_metrics(y_test, y_pred_test)
        }
        # Armazena as métricas de cada fold do treino do cross-validation apenas para o melhor modelo
        results[algo_name, scaler_name]["cv_scores"] = []
        fold_metrics = {}
        for fold in range(CV_FOLDS):
            fold_metrics[f"fold_{fold}_train"] = opt.cv_results_[f'split{fold}_train_score'][best_idx]
            fold_metrics[f"fold_{fold}_val"] = opt.cv_results_[f'split{fold}_test_score'][best_idx]
        results[algo_name, scaler_name]["cv_scores"].append(fold_metrics)

# Salvar variáveis para execução sem necessidade de novo treinamento
# Lista de variáveis que você deseja salvar para reuso posterior
vars_to_save = [
    'scalers', 'algo_configs', 'results', 'opt_res', 'N_ITER'
]

# Dicionário para armazenar as variáveis
vars_dict = {}

for var in vars_to_save:
    if var in globals():
        vars_dict[var] = globals()[var]

# Salvar em arquivo
joblib.dump(vars_dict, 'training_checkpoint.joblib')
print("Variáveis salvas em 'training_checkpoint.joblib'")


Executando HPO para: Linear - Minmax
 - Tempo estimado para Linear - Minmax: 24.71 segundos
Executando HPO para: Ridge - Minmax
 - Tempo estimado para Ridge - Minmax: 38.16 segundos
Executando HPO para: Lasso - Minmax
 - Tempo estimado para Lasso - Minmax: 41.36 segundos
Executando HPO para: ElasticNet - Minmax
 - Tempo estimado para ElasticNet - Minmax: 88.36 segundos
Executando HPO para: SVR - Minmax
 - Tempo estimado para SVR - Minmax: 133.59 segundos
Executando HPO para: KNN - Minmax
 - Tempo estimado para KNN - Minmax: 82.99 segundos
Executando HPO para: DecisionTree - Minmax
 - Tempo estimado para DecisionTree - Minmax: 112.71 segundos
Executando HPO para: RandomForest - Minmax
 - Tempo estimado para RandomForest - Minmax: 420.71 segundos
Executando HPO para: AdaBoost - Minmax
 - Tempo estimado para AdaBoost - Minmax: 314.65 segundos
Executando HPO para: GradientBoost - Minmax
 - Tempo estimado para GradientBoost - Minmax: 379.31 segundos
Executando HPO para: Bagging - Minmax
 - 

# 6 Treinamento dos modelos com otimização de hiperparâmetros

In [ ]:
import joblib

# Carregar variáveis salvas do arquivo 'training_checkpoint.joblib'
url_cp = 'https://raw.githubusercontent.com/fdamata/pucrj-machinelearning-mvp-ml/ded71860989b54bcf713791b2de1f388e643dd2f/training_checkpoint.joblib'
checkpoint = joblib.load(url_cp)

# Exemplo de acesso às variáveis carregadas:
scalers = checkpoint['scalers']
algo_configs = checkpoint['algo_configs']
results = checkpoint['results']
opt_res = checkpoint['opt_res']
N_ITER = checkpoint['N_ITER']

print("Variáveis carregadas de 'training_checkpoint.joblib':", list(checkpoint.keys()))

## 6.1 Desempenho computacional do treinamento dos modelos

In [ ]:
print(f"{'Algoritmo':<25} {'Scaler':<10} {'Iterações':<10} {'Early Stop?':<12} {'Tempo (s)':<10}")
print("-" * 75)
for (algo, scaler), res in opt_res.items():
    # Recupera o alias do algoritmo a partir do dicionário de configuração
    alias = algo_configs[algo]['alias'] if algo in algo_configs else algo
    n_iter = len(res['optimizer_result'].x_iters)
    elapsed = res['elapsed_time']
    early_stop = n_iter < N_ITER
    print(f"{alias:<25} {scaler:<10} {n_iter:<10} {'Sim' if early_stop else 'Não':<12} {elapsed:<10.2f}")

total_elapsed = sum(res['elapsed_time'] for res in opt_res.values())
print("-" * 75)
print(f"{'Tempo total (s):':<61}{total_elapsed:<10.2f}")

Importante observar que houve atuação da estratégia de parada prematura do treinamento de algoritmos em função de atingimento do critério de evolução da métrica.

In [ ]:
# Juntar todos os resultados de otimização em uma lista
optimizer_results = [res['optimizer_result'] for res in opt_res.values()]

# Calcular os limites globais do eixo y
all_func_vals = []
for res in optimizer_results:
	all_func_vals.extend(res.func_vals)
ymin = min(all_func_vals)
ymax = max(all_func_vals)
margin = 0.05 * (ymax - ymin) if ymax > ymin else 0.1
ymin_plot = ymin - margin
ymax_plot = ymax + margin

# Plotar todas as curvas de convergência em um único gráfico
fig, ax = plt.subplots(figsize=(10, 6))
plot_convergence(*optimizer_results, ax=ax)
ax.set_ylim(ymin_plot, ymax_plot)
ax.set_title('Convergência da Otimização Bayesiana (todos os modelos)', fontsize=14)
ax.set_xlabel('Iteração')
ax.set_ylabel('Score')
ax.grid(True, linestyle='--', alpha=0.5)
plt.show()

## 6.2 Desempenho das métricas de treino e teste dos modelos

In [ ]:
# Preparar dados para boxplot dos folds de treino, ponto do teste e linhas baseline e melhor teste global
# Encontrar o melhor modelo (menor erro para métricas de minimização, maior valor para maximização)

data = []
best_key = None
best_score = None

for (algo, scaler), result in results.items():
    # Adiciona os scores de validação dos folds
    for fold_score in result['cv_scores'][0]:
        if fold_score.startswith('fold_') and fold_score.endswith('_val'):
            data.append({'Algoritmo': algo_configs[algo]['alias'], 'Scaler': scaler, 'Score': result['cv_scores'][0][fold_score], 'Type': 'CV'})
    # Adiciona o score de teste final
    test_score = result['scores_test'][METRIC_TO_OPT]
    data.append({'Algoritmo': algo_configs[algo]['alias'], 'Scaler': scaler, 'Score': test_score, 'Type': 'Test'})

    score = result['scores_test'][METRIC_TO_OPT]
    if METRIC_TO_OPT in MINIMIZE_METRICS:
        if (best_score is None) or (score < best_score):
            best_score = score
            best_key = (algo, scaler)
    else:
        if (best_score is None) or (score > best_score):
            best_score = score
            best_key = (algo, scaler)

df_box = pd.DataFrame(data)

# Boxplot agrupado por scaler
boxplot_algo_grouped_by_scaler(df_box, METRIC_TO_OPT, ncols= len(scalers), figsize_per_plot=(6,4))



In [ ]:
# Plot 2: Influencia hiperparametro

for algo, res in opt_res.items():
    print(f"Algoritmo: {algo}")
    plot_objective(res['optimizer_result'], n_points=40, size=3)
    plt.suptitle('Impacto dos Hiperparâmetros')
    plt.tight_layout()
    plt.show()



In [ ]:
# Recuperar o melhor modelo considerando a métrica escolhida (METRIC_TO_OPT)
# Usar os resultados já armazenados no dicionário 'results' e a lista de scalers

# 2. Encontrar o melhor modelo (menor erro para métricas de minimização, maior valor para maximização)
best_key = None
best_score = None

for (algo, scaler), res in results.items():
    score = res['scores_test'][METRIC_TO_OPT]
    if METRIC_TO_OPT in MINIMIZE_METRICS:
        if (best_score is None) or (score < best_score):
            best_score = score
            best_key = (algo, scaler)
    else:
        if (best_score is None) or (score > best_score):
            best_score = score
            best_key = (algo, scaler)

# 3. Detalhar o melhor modelo, scaler e hiperparâmetros
if best_key is not None:
    best_algo, best_scaler = best_key
    best_params = results[best_key]['params']
    print(f"Melhor modelo considerando a métrica '{METRIC_TO_OPT}':")
    print(f"  Algoritmo: {best_algo}")
    print(f"  Scaler: {best_scaler}")
    print(f"  Score de teste ({METRIC_TO_OPT}): {best_score:.4f}")
    print(f"  Melhores hiperparâmetros encontrados:")
    for param, value in best_params.items():
        print(f"    {param}: {value}")
else:
    print("Não foi possível encontrar o melhor modelo.")

In [ ]:
# --- Avaliação do Melhor Modelo no Conjunto de Teste (Dados Não Vistos) ---

# Recuperar o melhor modelo e scaler do dicionário 'results' (já definido em célula anterior)
# O best_key, best_algo, best_scaler, best_params já foram definidos na célula anterior



# 1. Recuperar o melhor modelo e scaler
best_algo, best_scaler = best_key
best_params = results[best_key]['params']

# Encontrar o scaler correspondente
scaler_obj = None
for name, scaler in scalers:
    if name == best_scaler:
        scaler_obj = scaler
        break

# Encontrar a classe do modelo e os parâmetros default
model_class = None
module_name = None
default_params = None
for algo_name, config in algo_configs.items():
    if algo_name == best_algo:
        model_class = config['model_class']
        module_name = config['module']
        default_params = config['default_params']
        break

if scaler_obj is None or model_class is None or module_name is None:
    print("Erro: Não foi possível encontrar o modelo ou scaler correspondente.")
else:
    # Importar a classe do modelo dinamicamente
    import importlib
    model_cls = getattr(importlib.import_module(module_name), model_class)
    model_obj = model_cls(**default_params)

    # 2. Criar o pipeline com o melhor scaler e modelo
    pipeline = Pipeline([
        ('scaler', scaler_obj),
        ('regressor', model_obj)
    ])

    # 3. Definir os melhores hiperparâmetros encontrados
    final_params = {}
    for key, value in best_params.items():
        if not key.startswith('regressor__'):
            final_params[f'regressor__{key.replace("model__", "")}'] = value
        else:
            final_params[key] = value
    pipeline.set_params(**final_params)

    # 4. Treinar o pipeline com os dados de treino
    pipeline.fit(X_train, y_train)

    # 5. Fazer previsões no conjunto de teste
    y_pred_test = pipeline.predict(X_test)

    # 6. Calcular as métricas de avaliação
    r2_test = r2_score(y_test, y_pred_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    rmse_test = np.sqrt(mse_test)
    mae_test = mean_absolute_error(y_test, y_pred_test)

    # 7. Exibir os resultados
    print("="*80)
    print(f"Resultados do Melhor Modelo no Conjunto de Teste (X_test)")
    model_name_in_pipeline = pipeline.named_steps['regressor'].__class__.__name__
    print(f"Melhor scaler: {best_scaler}")
    print(f"Modelo: {model_name_in_pipeline}")
    print("="*80)

    metrics_data = [
        ["R² (R-squared)", f"{r2_test:.4f}"],
        ["MSE (Mean Squared Error)", f"{mse_test:.4f}"],
        ["RMSE (Root Mean Squared Error)", f"{rmse_test:.4f}"],
        ["MAE (Mean Absolute Error)", f"{mae_test:.4f}"]
    ]
    headers = ["Métrica", "Valor no Conjunto de Teste"]

    print(tabulate(metrics_data, headers=headers, tablefmt="grid"))
    print("="*80)

In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.base import clone
# from tabulate import tabulate


# 1. Recuperar o melhor modelo e scaler
best_algo, best_scaler = best_key
best_params = results[best_key]['params']

# Encontrar o scaler correspondente
scaler_obj = None
for name, scaler in scalers:
    if name == best_scaler:
        scaler_obj = scaler
        break

# Encontrar a classe do modelo e os parâmetros default
model_class = None
module_name = None
default_params = None
for algo_name, config in algo_configs.items():
    if algo_name == best_algo:
        model_class = config['model_class']
        module_name = config['module']
        default_params = config['default_params']
        break

if scaler_obj is None or model_class is None or module_name is None:
    print("Erro: Não foi possível encontrar o modelo ou scaler correspondente.")
else:
    # Importar a classe do modelo dinamicamente
    model_cls = getattr(importlib.import_module(module_name), model_class)
    model_obj = model_cls(**default_params)

    # Criar o pipeline com o melhor scaler e modelo
    final_pipeline = Pipeline([
        ('scaler', scaler_obj),
        ('regressor', model_obj)
    ])

    # Definir os melhores hiperparâmetros encontrados
    final_params = {}
    for key, value in best_params.items():
        if not key.startswith('regressor__'):
            final_params[f'regressor__{key.replace("model__", "")}'] = value
        else:
            final_params[key] = value
    final_pipeline.set_params(**final_params)

    # Retreinar o pipeline com todos os dados (X e y)
    print("="*80)
    print("Retreinando o melhor modelo com o dataset completo (X, y)")
    print("="*80)
    final_pipeline.fit(X, y)

    # Fazer previsões nos próprios dados de treinamento para avaliar o ajuste
    y_pred_full = final_pipeline.predict(X)

    # Calcular e exibir as métricas de desempenho no conjunto de dados completo
    r2_full = r2_score(y, y_pred_full)
    mse_full = mean_squared_error(y, y_pred_full)
    rmse_full = np.sqrt(mse_full)
    mae_full = mean_absolute_error(y, y_pred_full)

    metrics_data = [
        ["R² (R-squared)", f"{r2_full:.4f}"],
        ["MSE (Mean Squared Error)", f"{mse_full:.4f}"],
        ["RMSE (Root Mean Squared Error)", f"{rmse_full:.4f}"],
        ["MAE (Mean Absolute Error)", f"{mae_full:.4f}"]
    ]
    headers = ["Métrica", "Valor no Dataset Completo"]

    print(tabulate(metrics_data, headers=headers, tablefmt="grid"))
    print("="*80)

    # Salvar o modelo final em disco
    model_filename = f'modelo_final_{target}.joblib'
    joblib.dump(final_pipeline, model_filename)
    print(f"Modelo final salvo em '{model_filename}'")

In [ ]:


# --- Gráficos de Análise do Modelo Final ---

# 1. Calcular o erro
prediction_error = y - y_pred_full

# 2. Criar os subplots
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=(
        "Tendência do Erro de Predição (Real - Previsto)",
        "Comparativo: Valores Reais vs. Valores Previstos"
    )
)

# --- Gráfico 1: Tendência do Erro ---
fig.add_trace(
    go.Scatter(
        x=y.index,
        y=prediction_error,
        mode='lines',
        name='Erro',
        line=dict(color='indianred')
    ),
    row=1, col=1
)
# Adiciona uma linha em y=0 para referência de erro nulo
fig.add_hline(y=0, line_dash="dash", line_color="black", row=1, col=1)

# --- Gráfico 2: Real vs. Previsto ---
# Adiciona a linha dos valores reais
fig.add_trace(
    go.Scatter(
        x=y.index,
        y=y,
        mode='lines',
        name='Valor Real',
        line=dict(color='royalblue')
    ),
    row=2, col=1
)
# Adiciona a linha dos valores previstos
fig.add_trace(
    go.Scatter(
        x=y.index,
        y=y_pred_full,
        mode='lines',
        name='Valor Previsto',
        line=dict(color='darkorange', dash='dot')
    ),
    row=2, col=1
)

# 3. Atualizar o layout geral
fig.update_layout(
    height=700,
    title_text="Análise Gráfica do Desempenho do Modelo Final no Dataset Completo",
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Atualizar os títulos dos eixos
fig.update_yaxes(title_text="Erro (Real - Previsto)", row=1, col=1)
fig.update_yaxes(title_text="Valor da Variável Target", row=2, col=1)
fig.update_xaxes(title_text="Índice da Amostra", row=2, col=1)


fig.show()

In [ ]:
# --- Salvando e Carregando o Modelo Final ---

# 1. Definir o nome do arquivo para salvar o modelo
model_filename = f'modelo_final_{target}.joblib'

# 2. Salvar o pipeline treinado em um arquivo
print("="*80)
print(f"Salvando o modelo final em '{model_filename}'...")
joblib.dump(final_pipeline, model_filename)
print("Modelo salvo com sucesso!")
print("="*80)


# --- Demonstração de Carregamento e Uso ---

# 3. Carregar o modelo a partir do arquivo
print("\nCarregando o modelo para demonstração...")
loaded_model = joblib.load(model_filename)
print("Modelo carregado com sucesso.")

# 4. Usar o modelo carregado para fazer uma predição em um novo dado
#    (usaremos a primeira linha do dataset X como exemplo)
sample_data = X.head(1)
prediction = loaded_model.predict(sample_data)

print("\n" + "="*80)
print("Exemplo de predição com o modelo carregado:")
print(f"Dados de entrada (primeira linha do dataset):\n{sample_data.to_string()}")
print(f"\nValor Predito: {prediction[0]:.4f}")
print("="*80)

# Conclusão

O presente estudo demonstrou a importância de um processo rigoroso de análise exploratória, tratamento e pré-processamento dos dados para problemas de regressão em ambientes industriais. A partir do dataset de Pressão de Vapor da Nafta, foi possível identificar, tratar e justificar a remoção de outliers, realizar imputação de valores faltantes, criar variáveis derivadas relevantes (como a razão de refluxo), além de aplicar técnicas de transformação para redução de assimetrias e multicolinearidade.

A análise estatística e visual das variáveis, aliada ao conhecimento de domínio, permitiu selecionar os atributos mais representativos para a modelagem preditiva, reduzindo redundâncias e otimizando o conjunto de dados para futuros algoritmos de machine learning. As etapas de normalização e padronização garantiram que as diferentes escalas das variáveis não influenciassem indevidamente o desempenho dos modelos.

